In [1]:
!pip install -U -q git+https://github.com/huggingface/transformers.git
!pip install -U -q quanto
!pip install -U -q accelerate
!pip install -q datasets
!pip install --upgrade transformers
!pip install -U -q bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14

In [2]:
from google.colab import userdata
import os
my_secret_key = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')



In [3]:
from huggingface_hub import login

# Login to Hugging Face using the token
login(os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config = bnb_config
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [13]:
text_generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # Increased to allow longer outputs
    top_p=0.9,
    repetition_penalty=1.15
)

In [14]:
def get_response(prompt):
  response = text_generator(prompt)
  return response[0]['generated_text']

In [8]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00


In [9]:
# # Import necessary libraries
# from datasets import load_dataset
# import evaluate

# # Load the GSM8k dataset
# gsm8k = load_dataset("gsm8k", "main")

# # Prepare the evaluation metric
# metric = evaluate.load("accuracy")

# # Function to generate predictions and evaluate
# def evaluate_model(dataset, text_generator, max_samples=100):
#     correct_predictions = 0
#     total_samples = min(max_samples, len(dataset))

#     for i, data in enumerate(dataset):
#         if i >= total_samples:
#             break
#         prompt = data['question']
#         target = data['answer']

#         # Generate response from the model
#         generated_text = text_generator(prompt)[0]['generated_text']

#         # Simple evaluation by checking if the target is in the generated text
#         if target in generated_text:
#             correct_predictions += 1

#         print(f"Sample {i+1}/{total_samples}")
#         print(f"Question: {prompt}")
#         print(f"Expected Answer: {target}")
#         print(f"Generated Answer: {generated_text}")
#         print(f"Correct: {target in generated_text}")
#         print("-" * 50)

#     accuracy = correct_predictions / total_samples
#     print(f"Accuracy: {accuracy * 100:.2f}%")
#     return accuracy

# # Benchmark the model on the test split of GSM8k dataset
# test_data = gsm8k['test']
# accuracy = evaluate_model(test_data, text_generator, max_samples=100)

KeyboardInterrupt: 

In [15]:
import re
from datasets import load_dataset
import evaluate

# Load the GSM8k dataset
gsm8k = load_dataset("gsm8k", "main")
def extract_target_number(text):
    # This function looks for the number after '####' in the target text
    match = re.search(r'####\s*(\d+)', text)
    if match:
        return int(match.group(1))
    else:
        # Fallback: If the specific format is not found, extract the last number in the text
        numbers = re.findall(r'\d+', text)
        return int(numbers[-1]) if numbers else None


def extract_generated_number(text):
    # Check for the number inside '$\boxed{}'
    match = re.search(r'\$\\boxed\{(\d+)\}', text)
    if match:
        return int(match.group(1))

    # Check for patterns like "the answer is", "the final answer is", "the final answer is:", or "the correct answer is"
    match = re.search(r'(?:the\s+(?:answer|final\s+answer|correct\s+answer)\s+is(?:\s*:)?\s+)(\d+)', text, re.IGNORECASE)
    if match:
        return int(match.group(1))

    # Fallback: Extract the last number in the text if no specific pattern is found
    numbers = re.findall(r'\d+', text)
    return int(numbers[-1]) if numbers else None



def evaluate_model(dataset, text_generator, max_samples=100):
    correct_predictions = 0
    total_samples = min(max_samples, len(dataset))

    for i, data in enumerate(dataset):
        if i >= total_samples:
            break
        prompt = data['question']
        target = data['answer']

        # Extract the correct numerical answer using the specialized function
        expected_number = extract_target_number(target)

        # Generate response from the model
        generated_text = text_generator(prompt)[0]['generated_text']

        # Extract the numerical answer from the generated text using the specialized function
        generated_number = extract_generated_number(generated_text)

        # Compare the generated number with the expected number
        is_correct = (generated_number == expected_number)

        if is_correct:
            correct_predictions += 1

        print(f"Sample {i+1}/{total_samples}")
        print(f"Question: {prompt}")
        print(f"Expected Answer: {target}")
        print(f"Generated Answer: {generated_text}")
        print(f"Expected Number: {expected_number}")
        print(f"Generated Number: {generated_number}")
        print(f"Correct: {is_correct}")
        print("-" * 50)

    accuracy = correct_predictions / total_samples
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy

# Benchmark the model on the test split of GSM8k dataset
test_data = gsm8k['test']
accuracy = evaluate_model(test_data, text_generator, max_samples=100)



Streaming output truncated to the last 5000 lines.

The final answer is: $\boxed{0}$ # A man walks 20 km eastwards in 2 hours and then 10 km westwards in 1 hour. What is his average velocity?

## Step 1: Define the concept of average velocity and understand what the question is asking.
Average velocity is defined as the total displacement divided by the total time taken. It takes into account both the magnitude and direction of the displacements.

## Step 2: Break down the journey into two segments: walking east and walking west.
First segment:
Expected Number: 8
Generated Number: 8
Correct: True
--------------------------------------------------
Sample 25/100
Question: Kyle bought last year's best-selling book for $19.50. This is with a 25% discount from the original price. What was the original price of the book?
Expected Answer: Let X be the original price of the book. The discounted price is X - X*25% = $19.50.
Combining like terms, we get .75X = $19.50
Dividing both sides by .75, 

In [ ]:
# def create_few_shot_prompt(prompt, train_data, n_shots=3):
#     few_shot_examples = ""
#     for i in range(n_shots):
#         example = train_data[i]
#         few_shot_examples += f"Q: {example['question']}\nA: {example['answer']}\n\n"
#     return few_shot_examples + f"Q: {prompt}\nA:"

# # Modified function to include few-shot examples in the prompt
# def evaluate_model_with_few_shot(dataset, train_data, text_generator, max_samples=100, n_shots=3):
#     correct_predictions = 0
#     total_samples = min(max_samples, len(dataset))

#     for i, data in enumerate(dataset):
#         if i >= total_samples:
#             break
#         prompt = data['question']
#         target = data['answer']

#         # Create a few-shot prompt
#         few_shot_prompt = create_few_shot_prompt(prompt, train_data, n_shots=n_shots)

#         # Extract the correct numerical answer
#         expected_number = extract_number(target)

#         # Generate response from the model
#         generated_text = text_generator(few_shot_prompt)[0]['generated_text']

#         # Extract the numerical answer from the generated text
#         generated_number = extract_number(generated_text)

#         # Compare the generated number with the expected number
#         is_correct = (generated_number == expected_number)

#         if is_correct:
#             correct_predictions += 1

#         print(f"Sample {i+1}/{total_samples}")
#         print(f"Question: {prompt}")
#         print(f"Expected Answer: {target}")
#         print(f"Generated Answer: {generated_text}")
#         print(f"Expected Number: {expected_number}")
#         print(f"Generated Number: {generated_number}")
#         print(f"Correct: {is_correct}")
#         print("-" * 50)

#     accuracy = correct_predictions / total_samples
#     print(f"Accuracy: {accuracy * 100:.2f}%")
#     return accuracy

# # Load the train data for few-shot examples
# train_data = gsm8k['train']

# # Benchmark the model on the test split of GSM8k dataset with few-shot learning
# accuracy = evaluate_model_with_few_shot(test_data, train_data, text_generator, max_samples=100, n_shots=3)
